In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.transforms import Compose, Lambda
from torchvision import transforms
from torchvision.models.video import r2plus1d_18
from torchvision.io import read_video
from sklearn.model_selection import train_test_split
import os
import numpy as np
import torch
import torch.nn.functional as F

In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
class VideoDataset(Dataset):
    def __init__(self, root_dir, frame_count, resize, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.labels = []
        self.classes = os.listdir(root_dir)
        self.classes.sort()
        self.class_to_idx = {self.classes[i]: i for i in range(len(self.classes))}
        self.frame_count = frame_count
        self.resize = resize

        for idx, action_class in enumerate(self.classes):
            class_path = os.path.join(root_dir, action_class)
            for video_file in os.listdir(class_path):
                if video_file.endswith(('.mp4', '.avi')):
                    self.data.append(os.path.join(class_path, video_file))
                    self.labels.append(idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        video_path = self.data[idx]
        label = self.labels[idx]
        video, _, _ = read_video(video_path)

        # Resize and select a fixed number of frames
        video = self.process_video(video)

        if self.transform:
            video = self.transform(video)
        return video, label



    def process_video(self, video):
        # Assuming 'video' is a list of frames, each frame can be either a numpy array or a tensor of shape [height, width, channels]

        resized_frames = []
        for i in range(min(self.frame_count, len(video))):
            frame = video[i]

            # Convert numpy array to tensor
            if isinstance(frame, np.ndarray):
                frame = torch.from_numpy(frame).float()

            # Ensure frame has 3 channels (RGB)
            if frame.size(0) == 1:  # Grayscale to RGB
                frame = frame.repeat(3, 1, 1)

            # Resize frame
            frame = F.interpolate(frame.unsqueeze(0), size=self.resize, mode='bilinear', align_corners=False).squeeze(0)

            resized_frames.append(frame)

        # If the video is shorter than the required frame count, repeat the last frame
        while len(resized_frames) < self.frame_count:
            resized_frames.append(resized_frames[-1])

        # Stack frames
        video_tensor = torch.stack(resized_frames, dim=0)  # Shape: [frames, channels, height, width]

        # Permute to get the correct shape [channels, frames, height, width]
        video_tensor = video_tensor.permute(1, 0, 2, 3)

        return video_tensor


In [4]:

transform = transforms.Compose([
    # Add necessary transformations here (e.g., resizing, normalization)
])

# Load datasets
dataset = VideoDataset(root_dir='merged', transform=transform, frame_count=16, resize=(224, 224))
train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)

train_subset = Subset(dataset, train_indices)
test_subset = Subset(dataset, test_indices)

train_loader = DataLoader(train_subset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=1)


In [5]:
num_classes = len(dataset.classes)
model = r2plus1d_18()
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device("mps")
model.to(device)


VideoResNet(
  (stem): R2Plus1dStem(
    (0): Conv3d(3, 45, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (1): BatchNorm3d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(45, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
    (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2Plus1D(
          (0): Conv3d(64, 144, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (1): BatchNorm3d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(144, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        )
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
num_epochs = 10  # Set the number of epochs

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_batches = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        # inputs = inputs.permute(0, 4, 1, 2, 3)
        # inputs = inputs.permute(0, 2, 1, 3, 4)
        print(inputs.shape)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        total_batches += 1

    average_loss = running_loss / total_batches
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")



torch.Size([1, 120, 16, 224, 224])


RuntimeError: Given groups=1, weight of size [45, 3, 1, 7, 7], expected input[1, 120, 16, 224, 224] to have 3 channels, but got 120 channels instead